In [46]:
import whisperx as whisper
model=whisper.load_model("tiny")
result = model.transcribe("../audio/fold.wav",language='en',temperature=0.7,initial_prompt="prompt hun sir",condition_on_previous_text=True)


c:\Users\Ashhad\anaconda3\envs\p_39\lib\site-packages\whisperx\transcribe.py:83: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [47]:
import json
jj=result['segments']
with open("fold.json","w",encoding='utf-8') as f:
    json.dump(jj,f)

In [48]:
import json
with open("fold.json","r",encoding='utf-8') as f:
    cont=json.load(f)

In [49]:
import torch
if torch.cuda.is_available():
    device="gpu"
else:
    device="cpu"

In [50]:
model_a, metadata = whisper.load_align_model(language_code='en',device=device)

# align whisper output
result_aligned = whisper.align(cont, model_a, metadata, "../audio/fold.wav",device=device)

# print(result_aligned["segments"]) # after alignment
# print(result_aligned["word_segments"]) # after alignment

In [31]:
char_segments=[]
word_segments=[]

for x in range(len(result_aligned['segments'])):
    word_segments.append({"word-segments":result_aligned['segments'][x]['word-segments'].to_dict(orient='records')})
    char_segments.append({"char-segments":result_aligned['segments'][x]['char-segments'].to_dict(orient='records')})

In [36]:
for x in range(len(result_aligned['segments'])):
    result_aligned['segments'][x]['word-segments']=word_segments[x]
    result_aligned['segments'][x]['char-segments']=char_segments[x]

In [38]:
len(result_aligned['segments'])

13

In [18]:
import requests
import json
url = "http://localhost:8000/whisperX/"
# open the JSON file to upload
#aligned json is optional and must only be passed if u already have the json file

with open("../notebook/fold.json", "r") as input_file:
    data = json.load(input_file)
files = {'audio': ('../audio/fold.wav', open('../audio/fold.wav', 'rb'), 'audio/wav'),
         'aligned_json': ("../notebook/fold.json", json.dumps(data), "application/json")}
data = {'transcription': 'srt', 'model_name': 'tiny','translate':False,
        "device":"cpu",
        "language":"en","patience":1.0,"temperature":1.0,
        "condition_on_previous/_text":False,"temperature_increment_on_fallback":0.2,
        "compression_ratio_threshold":2.4,"logprob_threshold":-1.0,"no_speech_threshold":0.6
        }

response = requests.post(url, files=files, data=data)

response.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)